In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### DATA SOURCE: https://www.feiertagskalender.ch/ferien.php?geo=3539&jahr=2020&klasse=0&hl=en

In [5]:
def parse_data(year):
    print(year)

    url=f"https://www.feiertagskalender.ch/ferien.php?geo=3539&jahr={year}&klasse=0&hl=en"

    r = requests.get(url)
    soup=BeautifulSoup(r.text, "lxml")

    rows = soup.find("table").find_all("tr")[1:]

    data = []
    for row in rows:
        sub_rows=row.find_all("td")

        data.append({
            "start" : pd.to_datetime(sub_rows[0].text, dayfirst=True),
            "end" : pd.to_datetime(sub_rows[3].text.replace(" *", ""), dayfirst=True),
            "reason" : sub_rows[-2].text + " " + sub_rows[-1].text
        })

    return pd.DataFrame(data)

In [6]:
df = pd.concat((parse_data(year=year) for year in range(2020, 2024)), ignore_index=True)

2020
2021
2022
2023


In [18]:
df['reason'] = (
    df['reason'].str.strip()
    .replace(["Novemberferien", 
              "Tag der Republik Türkei", 
              "Feiertag der Jugend", 
              "Jugend und Sporttag", 
              "Feiertag der Nationalen Souveränität",
              "Nationale Souveränität und des Kindes", 
              "Festival of Breaking the Fast ('Id al Fitr)", 
              "Fest des Fastenbrechens", 
              "Novemberferien Provisional"],
              ["November vacation", 
               "Republic of Türkiye Day", 
               "holiday of youth", 
               "Youth and Sports Day", 
               "National Sovereignty Holiday",
               "National sovereignty and the child", 
               "Eid Holidays", 
               "Feast of the Breaking of the Fast", 
               "November vacation"])
    .replace(" Provisional", "", regex=True)
)

In [20]:
df.to_csv("tr_ist_school_holidays.csv", index=False)